In [ ]:
import torch

In [ ]:
A = torch.arange(-10,10,1)
A

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(A)

In [ ]:
plt.plot(torch.relu(A))

In [ ]:
def relu(x):
    return torch.maximum(torch.tensor(0), x)

In [ ]:
relu(A)

In [ ]:
def sigmoid(x):
    return (1 / ( 1 + torch.exp(-x)))

In [ ]:
sigmoid(A)

In [ ]:
plt.plot(sigmoid(A))

In [ ]:
from sklearn.datasets import make_blobs # https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html
from sklearn.model_selection import train_test_split


# Set the hyperparameters for data creation
NUM_CLASSES = 4
NUM_FEATURES = 2
RANDOM_SEED = 42


# 1. Create Multiclass data
X_blob, y_blob = make_blobs(n_samples=1000,
                            n_features = NUM_FEATURES,
                            centers=NUM_CLASSES,
                            cluster_std=1.5,
                            random_state=RANDOM_SEED)
# 2. Turn data into tensors
X_blob = torch.from_numpy(X_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.LongTensor)

# 3. Split into train and test

X_blob_train, X_blob_test, y_blob_train, y_blob_test = train_test_split(X_blob, y_blob, test_size=0.2, random_state=RANDOM_SEED)


# 4. Plot data (visualize)

plt.figure(figsize=(10,7))

plt.scatter(X_blob[:,0], X_blob[:,1], c=y_blob, cmap= plt.cm.RdYlBu)

In [ ]:
X_blob_test

# 8.2 Building a multi-class classification model in PyTorch

In [ ]:
# Craete device aganstic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# Build a multi-class classification model
from torch import nn 
class BlobModel(nn.Module):
    def __init__(self, input_features, output_features, hidden_units = 8):
        """Intializes multi-class classification model.
        Args:
            Input Features (int): Number of input feature to the model
            Output feature (int): Numb er of of output features (nuymber of output classes)
            hidden_units (int): Number of hidden units between layers, defaults 8 
            
            Return:
            
            Examples:
            """
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            #nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            #nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_features))
    def forward(self, x):
        return self.linear_layer_stack(x)
        
        

In [ ]:
X_blob_train.shape, y_blob_train.shape, X_blob_train.shape[1]

In [ ]:
len(y_blob_train.unique())

In [ ]:
# create an instance of BlobModel and send it to the target device
model_1 = BlobModel(input_features=X_blob_train.shape[1], output_features=len(y_blob_train.unique())).to(device)
model_1


In [ ]:
next(model_1.parameters()).device

In [ ]:
# Create a loss function of crossentropy
loss_fn = nn.CrossEntropyLoss()
# create a optimizer
optimizer = torch.optim.SGD(params = model_1.parameters(), 
                            lr = 0.1) # learning rate is a hyperparameter you can change

## 8.4  getting prediction probablilites for a multi-class pytorch model
In order to evaluate  and train and test our model, we need to convert our model's outputs (logits) to prediction probabilities and then to prediction labels.

Logits -> Pred probs -> Pred labels

In [ ]:
X_blob_train, X_blob_test, y_blob_train, y_blob_test = X_blob_train.to(device), X_blob_test.to(device), y_blob_train.to(device), y_blob_test.to(device)
model_1.eval()
with torch.inference_mode():
    y_logits = model_1(X_blob_test)

y_logits[:10]

In [ ]:
y_blob_test[:10]

In [ ]:
# convert our model's logit output to prediction probablities
y_pred_probs = torch.softmax(y_logits, dim=1)
print(y_logits[:5])
print(y_pred_probs[:5])

In [ ]:
y_pred = torch.argmax(y_pred_probs, dim=1)
y_pred[:5]


In [ ]:
y_blob_test[:5]

In [ ]:
y_pred_probs.argmax(dim=1)
# [ torch.sum(i).tolist() for i in y_pred_probs]

## 8.5 Creating a training loop and testing loop for a multi-class classification model

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()

    acc = (correct/len(y_pred)) * 100
    return acc

In [ ]:
y_blob_train.dtype

In [ ]:
# Fit the multi-class model to the data
torch.manual_seed(42)
torch.cuda.manual_seed(42)

X_blob_train, X_blob_test, y_blob_train, y_blob_test = X_blob_train.to(device), X_blob_test.to(device), y_blob_train.to(device), y_blob_test.to(device)
# Set the number of epochs
epochs = 100

for epoch in range(epochs):
    ### Training
    model_1.train()
    
    y_logits = model_1(X_blob_train)
    y_pred_prob = torch.softmax(y_logits, dim=1) #.argmax(dim=1)
    y_pred = torch.argmax(y_pred_prob, dim=1)

    loss = loss_fn(y_logits, y_blob_train)
    acc = accuracy_fn(y_true=y_blob_train, y_pred=y_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    ### Testing
    model_1.eval()
    with torch.inference_mode():
        test_logits = model_1(X_blob_test)
        test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
        test_loss = loss_fn(test_logits, y_blob_test)
        test_acc = accuracy_fn(y_true=y_blob_test, y_pred=test_pred)

    # print out what's happening
    if epoch % 10 == 0:
            print(f"Epoch: {epoch} | Loss: {loss:.5f} | Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f} | Test Accuarcy: {test_acc:.2f}%")
    
   
    

### 8.6 Making and evaluting predictions with a PyTorch multi-class model

In [ ]:
model_1.eval()
with torch.inference_mode():
    y_logits = model_1(X_blob_test)
y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)
y_pred

In [ ]:
y_blob_test

In [ ]:
## 4. Make predictions and evaluate the model

# Model isn't learning anything
import numpy as np
def plot_decision_boundary(model: torch.nn.Module, X: torch.Tensor, y: torch.Tensor):
    """Plots decision boundaries of model predicting on X in comparison to y.

    Source - https://madewithml.com/courses/foundations/neural-networks/ (with modifications)
    """
    # Put everything to CPU (works better with NumPy + Matplotlib)
    model.to("cpu")
    X, y = X.to("cpu"), y.to("cpu")

    # Setup prediction boundaries and grid
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))

    # Make features
    X_to_pred_on = torch.from_numpy(np.column_stack((xx.ravel(), yy.ravel()))).float()

    # Make predictions
    model.eval()
    with torch.inference_mode():
        y_logits = model(X_to_pred_on)

    # Test for multi-class or binary and adjust logits to prediction labels
    if len(torch.unique(y)) > 2:
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)  # mutli-class
    else:
        y_pred = torch.round(torch.sigmoid(y_logits))  # binary

    # Reshape preds and plot
    y_pred = y_pred.reshape(xx.shape).detach().numpy()
    plt.contourf(xx, yy, y_pred, cmap=plt.cm.RdYlBu, alpha=0.7)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("Train")
plot_decision_boundary(model_1, X_blob_train, y_blob_train) 

plt.subplot(1,2,2)
plt.title("Test")
plot_decision_boundary(model_1, X_blob_test, y_blob_test) 


## 9. A few more classification metrics... (to evaluate our classification model)

* Accuracy - out of 100 samples, how many does our model get right?
* Precision 
* Recall
* F1-score
* Confusion matrix
* Classification report

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()

    acc = (correct/len(y_pred)) * 100
    return acc

In [ ]:
(torch.eq(y_blob_test, y_pred).sum()/len(y_pred) *100).item()

In [ ]:
#!pip install torchmetrics

In [ ]:
import torchmetrics
from torchmetrics import Accuracy

In [ ]:
# Setup metric
torchmetric_accuracy = Accuracy(task="multiclass", num_classes=4).to(device)

# Calculate accuracy
torchmetric_accuracy(y_pred, y_blob_test)



In [ ]:
# Setup metric
torchmetric_precision = torchmetrics.Precision(task="multiclass", num_classes=4).to(device)

# Calculate precision
torchmetric_precision(y_pred, y_blob_test)

In [ ]:
# Setup metric
torchmetric_recall = torchmetrics.Recall(task="multiclass", num_classes=4).to(device)

# Calculate precision
torchmetric_recall(y_pred, y_blob_test)

In [ ]:
# Setup metric
torchmetric_f1score = torchmetrics.F1Score(task="multiclass", num_classes=4).to(device)

# Calculate precision
torchmetric_f1score(y_pred, y_blob_test)

In [ ]:
# Setup metric
torchmetric_confusion = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=4).to(device)

# Calculate precision
torchmetric_confusion(y_pred, y_blob_test)

In [ ]:
auroc = torchmetrics.AUROC(task="multiclass", num_classes=4)
auroc(test_logits, y_blob_test)

In [ ]:
# # Setup metric
# torchmetric_confusion = torchmetrics.ClassificationReport(task="multiclass", num_classes=4).to(device)

# # Calculate precision
# torchmetric_confusion(y_pred, y_blob_test)

In [ ]:
from sklearn.metrics import classification_report

# Compute classification report
report = classification_report(y_pred.cpu(), y_blob_test.cpu(), target_names=['class_0', 'class_1', 'class_2', 'class_3'])  # Modify target_names as needed
print(report)

In [ ]:
#!pip install ignite

In [ ]:
torch.tensor([9], device=device)